# Mobile Robotics Project - Final Report

Group Members:
- ESCOYEZ, Antoine Jacques Richard, 335564
- POUSSIN, Jean-Baptiste Marie Alexandre, 303127
- KOCKISCH, Matthias Hugues Jörg, 303000
- STUBER, Lukas, 289304

Professor:

Prof. MONDADA, Francesco


EPFL, 11.12.2022
Course MICRO-452 Basics of Mobile Robotics

# Project Description
The goal of the project in the Basics of Mobile Robotics class was the control of a Thymio robot, a two wheeled robot. The environment of the robot is a map designed by the group members. On the map there need to be obstacles into which the robot cannot/must not drive. According to the chosen map and the obstacles present on the map a global path from the start to the goal needs to be calculated using computer vision (via a camera ?????). The start is the initial location of the Thymio, the goal can be chosen by the user. On its path to the goal the robot needs to be able to detect and avoid unforeseen obstacles, that are added by the user. For better tracking of the path some kind of filtering must be used that improves the state estimation of the robot. The camera image serves as measurement for the filter.

# Detailed Description of our Choices
Our group chose the map to be a parking space with two rows of parking lots. The yellow lines marking the edges of the parking lots are the fixed obstacles used in the global path planning. The robot is neighter allowed to cross the yellow lines nor leave the map. The camera is fixed above the map and oversees the entire situation (robot, lines, free spaces, occupied spaces). To facilitate the detection of the position and orientation of the robot in the images we sticked two coloured stickers on the top side of the robot. The left sticker is green, the right one is blue. The computer vision algorithm uses these two points to determine the center of the wheels axis and the orientation of the robot. The goal is a free parking lot indicated by a red paper square. The unknown obstacles (unknown to/ignored by the computer vision) are <font color='red'>????</font>
. The estimated state correction is done with the help of a Kalman filter whose measurement is the location of the Thymio in the camera images.


<img src="Map.jpg" width="300">

# Computer Vision
We use a *AUKEY 1080P - 30 pfs* camera as the optical sensor for the computer vision. It is fixed on a stand (a lamp) and sees the entire map from above. Its position does not change during the operation of the robot.

To enable a clear and stable detection of the objects on the map a series of adjustments must be done before the object detection. The image processing is done in HSV representation of the image. The procedure is automatically started when the main function is executed. It includes:
- Adjust the luminosity <font color='red'>????</font>
- Detection of the outer borders of the map
- Adjust the red upper and lower thresholds <font color='red'>????</font>
- Adjust the green upper and lower thresholds <font color='red'>????</font>
- Adjust the blue upper and lower thresholds <font color='red'>????</font>
- Adjust the yellow upper and lower thresholds <font color='red'>????</font>

<font color='red'>Describe the steps of the computer vision to find the colorized map.</font>

The computer vision algorithm abstracts the picture of the map leaving only the relevant objects (map surface, robot (green and blue spot), obstacle lines, goal) in full colors (rgb values eighter 255 or 0).

# Global Path Planning
The global path planning is done using the A* algorithm, as seen in Exercise 5 of the course. It uses the simplified map generated by the computer vision part as input. The typical size of this image is <font color='red'>????</font>. Obviously, this is far too large for the A* algorithm to finish in an acceptable time. To reduce the size of the image a convolutional filter is applied to the pixels. Its size is can be chosen by the user and is represented by the variable "kernel". The filter devides the image from the vision part into patches of size 2*kernel+1 by 2*kernel+1. The output pixel is 0 if:
- all the pixels in the patch are black (free space)
- at least one of the pixels is green or blue (robot position)
- at least one of the pixels is red (goal position)
The output is 1 if:
- at least one of the pixels is yellow (obstacle line)
The result of this filtering is a map of typical size 60 by 120 pixels (depending on the size of the input map) where the free, start and goal space is 0 and the obstacle space is 1.

This reduced map and the start and goal positions are fed into the A* algorithm that calculates the optimal global path from the start to the goal. The calculated path is a series of coordinates in the reduced map format. To serve as directions for the Thymio the coordinates need to be resized to the original map (multiplied by the reduction factor 2*kernel+1).

# Motion Control
Input: path (series of coordinates)
Output: speed of the wheels

The class ThymioControl in the file ``control.py`` solely handles the interactions between the PC and the Thymio through the ``tdmclient`` library, and contains the related higher-level algorithms for path following and obstacle avoidance. It thus controls the speeds of the robot’s wheels depending on the robot coordinates given by the Kalman filtering module, and then sends the speeds it calculated back to the latter.

```python
def __init__(self):
    self.client = ClientAsync()
    self.node = aw(self.client.wait_for_node())
    aw(self.node.lock())
    self.position = [0,0]
    self.angle = 0
    self.speed_target = (0,0)
# path following
    self.stop_planned = False
    self.path = [(0,0)]
    self.path_index = 0
# obstacle avoidance
    self.proxs = np.zeros(5)
    self.obst_direction = 0
# timers
    self.stop_timer = Timer(0, self.stop)
    self.move_timer = RepeatedTimer(MOVE_INTERVAL, self.navigation)
```

The function ``navigation()``, controlling both path following and obstacle avoidance, is called by the RepeatedTimer ``move_timer`` at a period ``MOVE_INTERVAL``. This function first reads the values of the proximity sensors of the Thymio, and then starts either an obstacle avoidance step if a significant value is read in fron tin the robot, or a path following step if not. The timer interval ``MOVE_INTERVAL`` is set to ``0.21`` seconds to leave time for both fetching of the sensor values and setting new motor values, both having a set duration of 100 milliseconds. ``move_timer`` is started by the function ``follow_path()``, which is called from the main program.

```python
def navigation(self):
    self.get_prox()
    if np.any(self.proxs > PROX_THRESHOLD):
        self.avoid_obstacles()
    else: self.move_to_goal()

def follow_path(self):
    self.move_timer.start()
```

The most basic movement function is ``move(l_speed, r_speed)`` which simply sends the desired speeds to the Thymio and records the current speed-target. This communication with the robot has a standard duration of 100 milliseconds, which makes an efficient PID controller running on the computer impossible. A PID running on the Thymio itself was also not desirable due to the frequent sending of observed positions and fetching of estimated positions, requiring event-listeners on both machines and each taking up additional 0.1 seconds. Running a PID and odometry on the Thymio itself is further impeded by the robot's inability to calculate with floating point values (running odometry on the computer would require speed-updates at a very high frequency, which is completely impossible with the communication delay).

We thus decided to greatly reduce the amount of communications by setting a constant speed for all movements and estimating its duration; the commands sent to the robot are then the initial start of the movement, and the stopping command after the calculated duration. In this case, no (relevant) code is flashed onto the Thymio.

```python
def move(self, l_speed, r_speed=None):
    if r_speed is None: r_speed = l_speed
    aw(self.node.set_variables({"motor.left.target": [int(l_speed)],"motor.right.target": [int(r_speed)]}))
    self.speed_target = (l_speed, r_speed)

```

why no pid

### Path following
The simple path following algorithm function ``move_to_goal()`` can be reduced to the following pseudo-code:

<pre>
calculate distance to goal  
if distance is bigger than DIST_TOL:  
    calculate angle to goal
    if angle is bigger than ANGLE_TOL:
        <i>re-orient robot</i>
    else:
        <i>move robot</i>
else:
    increment goal index in path array
    if index surpasses the length of the path:
        stop the movement timer
</pre>

The distance threshold ``DIST_TOL`` determines the point at which a goal is reached, while the angle threshold ``ANGLE_TOL`` triggers a re-orientation of the robot on its way to a goal. Their values were set to 10 millimeters and 0.1 rad = 5.73° respectively, which allows precise enough movement and good path following without any issues.

```python
def move_to_goal(self):
    if self.stop_planned: return
    goal = self.path[self.path_index]
    dist = math.sqrt((goal[0] - self.position[0])**2 + (goal[1] - self.position[1])**2)
    if dist > DIST_TOL:
        angle = (math.atan2(goal[1] - self.position[1], goal[0] - self.position[0]) - self.angle + math.pi) % (2*math.pi) - math.pi
        if abs(angle) > ANGLE_TOL:
            direction = 1 if angle > 0 else -1 # 1 = turn left, -1 = turn right
            t = abs(angle)*WHEEL_DIST / (2*STANDARD_SPEED*SPEED_TO_MMS) - 0.1 # remove the await delay of move()
            self.timed_move(direction*STANDARD_SPEED, -direction*STANDARD_SPEED, t*TIME_FACTOR) # prevent overshoots
        else:
            t = dist / (STANDARD_SPEED*SPEED_TO_MMS) - 0.1 # remove the await delay of move()
            self.timed_move(STANDARD_SPEED, STANDARD_SPEED, t*TIME_FACTOR) # prevent overshoots
    else:
        self.path_index += 1
        if self.path_index >= len(self.path): self.end_path()
```

Both italicised lines represent a movement of the robot. For these functions, the movement duration is first estimated using the real-world speed conversion constant ``SPEED_TO_MMS``, which has been experimentally found to be $0.32\:(mm/s)^{-1}$ (at our chosen speed of ``STANDARD_SPEED = 200``, the Thymio moves 32 centimeters in 5 seconds). The time is first multiplied by ``TIME_FACTOR = 0.75`` to prevent any overshoots of the goal coordinates; we prefer another short move forward instead of a slow 180°-rotation of the robot to correct the remaining difference to the goal.
The final move duration is used to program ``stop_timer``, which simply stops the robot and resets the boolean ``stop_planned`` to allows re-entry into ``move_to_goal()``. Since the ``stop()`` function accesses the Thymio's motor-targets over 100 milliseconds, this delay is subtracted from the duration. The timer is programmed by the function ``timed_move()``, which also sets ``stop_planned`` that prohibits any new command from ``move_to_goal()``.

```python
self.stop_timer = Timer(0, self.stop)

def stop(self):
    self.stop_planned = False
    self.move(0)
    
def timed_move(self, l_speed, r_speed, time):
    self.stop_planned = True
    self.stop_timer = Timer(time, self.stop)
    self.move(l_speed, r_speed)
    self.stop_timer.start()
```
### Obstacle avoidance
In case an obstacle is detected by the front proximity sensors, ``navigation()`` calls ``avoid_obstacles()``.

# Kalman Filter
Input: position and orientation of the robot seen from the camera
    estimated state
Output: corrected estimated state

# Local obstacles avoidance
Input: prox measurements
Output: speed of the wheels

# Main Function
bla bla, maybe talk about the timers

# Conclusion
conclude conclude

In [ ]:
from __future__ import print_function
import cv2 as cv
from parking_segmentation import *
from color_segmentation import *
from color_centroids import *
from discretize_map import *
from control import ThymioControl
from Kalman import Kalman
from constants import *
from RepeatedTimer import RepeatedTimer

# IMAGE PROCESSING
id_camera = 1
##[Parking segmentation]
corners, destination_corners = set_parking_limits(id_camera)
##[Color segmentation]
segmentation, refined_color_dict_HSV, kernels, openings = get_color_mask(id_camera, corners, destination_corners, real_size=(NOMINAL_AREA_LENGTH, NOMINAL_AREA_WIDTH))
cv.namedWindow("Segmentation Result")
cv.imshow("Segmentation Result", segmentation)
key = cv.waitKey(0)
cv.destroyWindow("Segmentation Result")
##[Thymio and objective localization]
centroids = {'goal': (0, 0), 'thymio': (0, 0), 'green': (0, 0), 'blue': (0, 0)}
theta_thymio = 0
localization = None

kalman = Kalman()
thymio = ThymioControl()

def compute_centroids():
    global centroids, theta_thymio, localization
    centroids, theta_thymio, localization = get_centroids(id_camera, corners, destination_corners, refined_color_dict_HSV, kernels, openings, prev_centroids=centroids, real_size=(NOMINAL_AREA_LENGTH, NOMINAL_AREA_WIDTH), real_time=False)
    #print("centroids at", centroids['thymio'][0], centroids['thymio'][1])
    #print("thymio angle at ", theta_thymio)
    kalman.state_correct(np.array([centroids['thymio'][0], centroids['thymio'][1], theta_thymio]))
    thymio.position = (kalman.x[0, 0], kalman.x[1, 0])
    thymio.angle = kalman.x[2, 0]

def odometry():
    kalman.state_prop(thymio.speed_target)
    thymio.position = (kalman.x[0, 0], kalman.x[1, 0])
    thymio.angle = kalman.x[2, 0]
    #print(thymio.position[0], thymio.position[1], thymio.angle*180/math.pi)

# def plot_localization():
#     global localization
#     cv.namedWindow("Localization Result")
#     cv.imshow("Localization Result", localization)
#     key = cv.waitKey(30)
#     cv.destroyWindow("Localization Result")

# initialise position and path
compute_centroids()
kalman.set_state((centroids['thymio'][0], centroids['thymio'][1], theta_thymio))
thymio.position = (centroids['thymio'][0], centroids['thymio'][1])
thymio.angle = theta_thymio
path = discretize_map(segmentation, centroids)
thymio.set_path(path)

# start updating position and follow path
image_timer = RepeatedTimer(1.5, compute_centroids)
odometry_timer = RepeatedTimer(ODOMETRY_INTERVAL, odometry)
image_timer.start()
odometry_timer.start()
thymio.follow_path()